## Session State encoder using LSTM Autoencoder

We used LSTM netural network to create an autoencoder that will reduce the dimension of session which contains last 5 played songs features and action between those songs (5x9 matrix) to a smaller vector of 1x9.

#### Importing necessary libraries for this notebook

In [1]:
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.layers import Input, Dense
from keras.models import Model
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


Loading the latent representation of each track from a csv.

In [22]:
latent_track_features_df = pd.read_csv("latent_track_features.csv");
latent_track_features_df.head()

,track_id,0,1,2,3,4,5,6,7
0,t_f9b9a072-8dae-4816-bfd6-d91fc75a5744,0.085536,0.023943,0.125375,-0.145933,0.449010,-0.106019,-0.190885,0.282416
1,t_7f435320-5d3b-497d-9346-0c97429af2b9,0.152102,-0.118725,-0.168167,0.291125,0.540807,0.042686,-0.288468,0.130090
2,t_949441c4-4920-4eed-b929-2d67d37b7bd9,0.056647,-0.077121,0.052887,-0.076303,0.644329,-0.148826,-0.292301,0.208705
3,t_63eae5ac-744a-46ee-b719-0db8e70d6f7f,0.120041,-0.071038,-0.012514,-0.061835,0.621581,-0.082328,-0.210929,0.185848
4,t_3c2c29b5-c738-42c0-83e1-ee0e88ac834d,0.075071,0.094992,0.372702,0.001345,0.534761,-0.017185,-0.420036,0.035725


Converting it into a dictionary for easy lookup.

In [24]:
latent_track_features_dict = latent_track_features_df.set_index('track_id').T.to_dict('list')

Reading session data from session_track_reward_data.csv created during sampling process.

In [26]:
session_df = pd.read_csv("session_track_reward_data.csv");
session_df.head()

,session_id,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,53_0068dc30-11ee-4417-b045-396c627ef013,t_6131a810-b1c3-4be9-92c9-54105e775151,1,t_309812b2-11d2-47ed-90fb-c0dbf25b0866,1,t_34871798-7470-4bf7-85a4-e0c8e277a2d3,1,t_2c06c4b3-f2a4-4ccc-b673-b9943aa7e1e3,0,t_94e173bf-9244-4c4b-b9a8-2a735a3d6ccf,...,t_7c13b9d7-39b7-48e0-828f-6c920ac6535f,1,t_17ee7a20-975e-4785-8e31-10d0da37457b,0,t_6e6b433c-59da-49bc-9bdf-ddc122928277,1,t_6e6b433c-59da-49bc-9bdf-ddc122928277,0,t_309812b2-11d2-47ed-90fb-c0dbf25b0866,0
1,53_006d0a57-11b2-42dd-98b6-625bb28283f6,t_8c1a8a18-7181-4364-bca4-94b97e5dc2d8,0,t_665cc6c4-7745-47bc-bb65-11eeb33e1ecc,0,t_a646aa9c-0c00-45a5-aa53-cb428e171251,1,t_69ef057b-3e0c-469e-a71a-4e348f6b6aed,0,t_2d9aef46-0e33-479f-ace4-019793ad5ea5,...,t_33831664-66b6-4590-9939-052d3c8c4b11,0,t_13ceba8d-4d30-44c7-a80d-9f75adca2432,0,t_6966e4af-fbdc-4e95-ab49-6af31c95f37a,0,t_26eff975-cb7e-4507-a3b6-278b526584bc,0,t_68613f0d-1617-4bd6-aacb-372a79ddd770,1
2,53_038a6aaa-7d85-4bab-bd25-2f69a3eb12b6,t_6838642c-b80e-4b0b-b18f-5389fb74d00f,0,t_d1f175e8-a157-411c-9447-d48615e71ed0,0,t_da1fef49-47b5-4236-842a-36347cb78460,0,t_e000b439-a7e6-4e85-84b7-4e3d54742d60,0,t_d55fd565-fe40-4442-b8f0-393e9222d23e,...,t_88e8671d-e6ee-48dc-b610-6d1cad2657cf,1,t_b7954b03-6717-42f4-a00d-fcaa26ac7f90,1,t_0337a921-6153-4c8b-8c27-679a3796dfc8,1,t_93cccc0a-8dcb-4697-b501-dc9056309abc,1,t_bf81cf32-8335-457d-a867-5fbaca8d56b9,1
3,53_040eeedd-a7ce-4229-a1c0-17e510ea9f76,t_d9b73bdb-eecf-4c8c-9f85-c2bd3bab8d33,0,t_4c316d3c-ab2e-4943-b0cd-c59a0c7480b4,0,t_d9b73bdb-eecf-4c8c-9f85-c2bd3bab8d33,0,t_59647f23-dd08-45d1-8eeb-03fdd9c1b718,0,t_ad62139c-561c-47d9-9481-744b570d74e7,...,t_d1f58de5-01c0-4bda-962c-62fed400e2f4,0,t_18b618d9-ad3e-4d24-8e6e-8e3c2c502ded,0,t_0db93ef0-06e8-460a-852a-9cf2752ae224,1,t_675dd421-0c06-4914-8beb-329eff278481,0,t_ac6deb7f-8a35-4db3-840f-4f06a7a1aab0,1
4,53_0546a593-43e6-4139-af76-bfd03c6da8fa,t_66c7eb2a-fa7e-4c56-ad55-3ee477cba71e,0,t_5704cdc3-c455-440c-868c-6a880b4c4476,0,t_6c743c12-884b-4164-b80b-895c17e700f9,1,t_3d0626fc-bf86-45e0-a61d-c49250b52771,1,t_0ba77058-5e81-4696-9bd4-c21cff3f3e82,...,t_b3c6727e-3253-4c10-a6d2-a3227f6bdc74,1,t_9d410287-c57c-4513-af4b-c0f0ea046889,1,t_8223208a-2bbf-4433-8a8c-9b8b7be92ade,1,t_01b5c742-7e2f-402f-9f54-1b4f6c04af95,1,t_821c2943-624e-4ba3-ace1-360a4c4b6e35,1


In [45]:
session_dict = session_df.set_index('session_id').T.to_dict('list')

Below function creates a matrix representation of a state in a session. A state holds the information of last 5 songs played along with the action between songs. So, this function takes input as last 5 songs and actions, and return a 5x9 matrix that represents that state.

In [72]:
def construct_state(track_and_reward_list):
    state_matrix = []
    for i in range(5):
        latent_track = latent_track_features_dict[track_and_reward_list[2*i]].copy()
        latent_track.append(track_and_reward_list[2*i + 1])
        state_matrix.append(latent_track)
    np_state_matrix = np.array(state_matrix)
    return np_state_matrix

In [73]:
state_list = []
for session_id in session_dict:
    track_and_reward_list = session_dict[session_id]
    for i in range(15):
        state = construct_state(track_and_reward_list[2*i:2*i + 10])
        state_list.append(state)

In [74]:
len(state_list)

1494900

In [75]:
len(session_dict)

99660

## LSTM Autoencoder

We used LSTM autoencoder because the data is sequential.

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [77]:
np_state_list = np.array(state_list)

In [95]:
print(np_state_list.shape)
timesteps = np_state_list.shape[1]
n_features = np_state_list.shape[2]

(1494900, 5, 9)


In [110]:
model = Sequential()
model.add(LSTM(18, activation='tanh', input_shape=(timesteps, n_features), return_sequences=True))
model.add(LSTM(9, activation='tanh', return_sequences=False))
model.add(RepeatVector(timesteps))
model.add(LSTM(9, activation='tanh', return_sequences=True))
model.add(LSTM(18, activation='tanh', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 5, 18)             2016      
_________________________________________________________________
lstm_18 (LSTM)               (None, 9)                 1008      
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 5, 9)              0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 5, 9)              684       
_________________________________________________________________
lstm_20 (LSTM)               (None, 5, 18)             2016      
_________________________________________________________________
time_distributed_7 (TimeDist (None, 5, 9)              171       
Total params: 5,895
Trainable params: 5,895
Non-trainable params: 0
____________________________________________________

In [111]:
# fit model
model.fit(np_state_list, np_state_list, epochs = 50, batch_size = 128, use_multiprocessing = True)

Epoch 1/50
1494900/1494900 [==============================] - 128s 86us/step - loss: 0.0103
Epoch 2/50
1494900/1494900 [==============================] - 129s 86us/step - loss: 0.0068
Epoch 3/50
1494900/1494900 [==============================] - 125s 83us/step - loss: 0.0059
Epoch 4/50
1494900/1494900 [==============================] - 119s 80us/step - loss: 0.0056
Epoch 5/50
1494900/1494900 [==============================] - 119s 79us/step - loss: 0.0054
Epoch 6/50
1494900/1494900 [==============================] - 120s 80us/step - loss: 0.0053
Epoch 7/50
1494900/1494900 [==============================] - 120s 80us/step - loss: 0.0051
Epoch 8/50
1494900/1494900 [==============================] - 119s 80us/step - loss: 0.0049
Epoch 9/50
1494900/1494900 [==============================] - 120s 80us/step - loss: 0.0048
Epoch 10/50
1494900/1494900 [==============================] - 120s 81us/step - loss: 0.0047
Epoch 11/50
1494900/1494900 [==============================] - 119s 80us/step -

In [112]:
encoder = Model(inputs=model.inputs, outputs=model.layers[1].output)

In [113]:
import pickle
pickle.dump(encoder, open('pickle/session_state_encoder.sav', 'wb'))